In [2]:
pip install transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 84.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 30.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 98.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 78.1 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import torch
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from sklearn.metrics import classification_report
from scipy.special import softmax

#veri hazırlama kısmı
def load_data(file_path):
    data = []
    with open(file_path, 'r') as f:
        for line in f:
            if line.strip() == '':
                continue
            text = line.strip()
            aspect = next(f).strip()

            sentiment = next(f).strip()
            data.append({'text': text, 'sentiment': sentiment, 'aspect': aspect})

    texts = [d['text'] for d in data]
    sentiments = [d['sentiment'] for d in data]
    aspects = [d['aspect'] for d in data]

    for i, text in enumerate(texts):
        texts[i] = text.replace("$T$", sentiments[i])

    sentiment_dict = {'Positive': 2, 'Neutral': 1, 'Negative': 0}
    sentiments = [sentiment_dict[sentiment] for sentiment in sentiments]
    data_dict = {'text': texts, 'aspect': aspects, 'sentiment': sentiments}
    df = pd.DataFrame(data_dict)

    return df

train_df = load_data('restaurants_train_turkish.xml.dat')
test_df = load_data('restaurants_test_turkish.xml.dat')

# BERT Tokenizer'ı yükleme
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

# Verilerinizi BERT modeline uygun hale getirme
def prepare_data(dataframe):
    input_ids = []
    attention_masks = []
    labels = []

    for index, row in dataframe.iterrows():
        encoded_dict = tokenizer.encode_plus(
            row['text'],
            add_special_tokens=True,
            max_length=64,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt',
        )
        input_ids.append(encoded_dict['input_ids'])
        attention_masks.append(encoded_dict['attention_mask'])
        labels.append(row['sentiment'])

    input_ids = torch.cat(input_ids, dim=0)
    attention_masks = torch.cat(attention_masks, dim=0)
    labels = torch.tensor(labels)

    return input_ids, attention_masks, labels

train_input_ids, train_attention_masks, train_labels = prepare_data(train_df)
test_input_ids, test_attention_masks, test_labels = prepare_data(test_df)

# Veri yükleyicilerini oluşturma
batch_size = 32

train_data = TensorDataset(train_input_ids, train_attention_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

test_data = TensorDataset(test_input_ids, test_attention_masks, test_labels)
test_sampler = SequentialSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

def train_model(model, train_dataloader, epochs=4):
    optimizer = AdamW(model.parameters(), lr=2e-5, eps=1e-8)
    for epoch in range(epochs):
        total_loss = 0
        model.train()
        for step, batch in enumerate(train_dataloader):
            b_input_ids = batch[0]
            b_input_mask = batch[1]
            b_labels = batch[2]
            model.zero_grad()
            outputs = model(b_input_ids,
                            token_type_ids=None,
                            attention_mask=b_input_mask,
                            labels=b_labels)
            loss = outputs.loss
            logits = outputs.logits
            total_loss += loss.item()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
        avg_train_loss = total_loss / len(train_dataloader)

# BERT modelini yükleme
model = BertForSequenceClassification.from_pretrained(
    "bert-base-multilingual-cased",
    num_labels=3,
    output_attentions=False,
    output_hidden_states=False,
)

# Modeli eğitimi
train_model(model, train_dataloader)

def evaluate_model(model, dataloader):
    model.eval()
    predictions , true_labels = [], []
    for batch in dataloader:
        b_input_ids, b_input_mask, b_labels = batch
        with torch.no_grad():
            outputs = model(b_input_ids,
                            token_type_ids=None,
                            attention_mask=b_input_mask)
        logits = outputs.logits
        logits = logits.detach().numpy()
        label_ids = b_labels.numpy()
        predictions.append(logits)
        true_labels.append(label_ids)
    predictions = [softmax(logits, axis=1) for logits in predictions]
    predictions = np.concatenate(predictions, axis=0)
    predictions = np.argmax(predictions, axis=1)
    true_labels = np.concatenate(true_labels, axis=0)
    print(classification_report(true_labels, predictions))

# Eğitim verilerini kullanarak modelinizin performansını değerlendirme
evaluate_model(model, train_dataloader)

# Test verilerini kullanarak modelinizin performansını değerlendirme
evaluate_model(model, test_dataloader)

# Hiperparametre optimizasyonu
learning_rates = [2e-5, 3e-5, 5e-5]
epochs = [2, 3, 4]

best_acc = 0
best_lr = None
best_epoch = None

for lr in learning_rates:
    for epoch in epochs:
        # Modeli yeniden yükle
        model = BertForSequenceClassification.from_pretrained(
            "bert-base-multilingual-cased",
            num_labels=3,
            output_attentions=False,
            output_hidden_states=False,
        )

        # Modeli eğitin
        train_model(model, train_dataloader, epochs=epoch)

        # Modeli değerlendir
        model.eval()
        predictions , true_labels = [], []
        for batch in test_dataloader:
            b_input_ids, b_input_mask, b_labels = batch
            with torch.no_grad():
                outputs = model(b_input_ids,
                                token_type_ids=None,
                                attention_mask=b_input_mask)
            logits = outputs.logits
            logits = logits.detach().numpy()
            label_ids = b_labels.numpy()
            predictions.append(logits)
            true_labels.append(label_ids)
        predictions = [softmax(logits, axis=1) for logits in predictions]
        predictions = np.concatenate(predictions, axis=0)
        predictions = np.argmax(predictions, axis=1)
        true_labels = np.concatenate(true_labels, axis=0)

        # Performansı kaydedilmesi
        acc = (predictions == true_labels).mean()
        if acc > best_acc:
            best_acc = acc
            best_lr = lr
            best_epoch = epoch

print(f"En iyi doğruluk: {best_acc}")
print(f"En iyi öğrenme oranı: {best_lr}")
print(f"En iyi epoch sayısı: {best_epoch}")

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       504
           1       1.00      1.00      1.00       111
           2       1.00      1.00      1.00       770

    accuracy                           1.00      1385
   macro avg       1.00      1.00      1.00      1385
weighted avg       1.00      1.00      1.00      1385

              precision    recall  f1-score   support

           0       0.98      1.00      0.99        46
           1       1.00      1.00      1.00         6
           2       1.00      0.99      0.99        94

    accuracy                           0.99       146
   macro avg       0.99      1.00      0.99       146
weighted avg       0.99      0.99      0.99       146



Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual

In [3]:
import pandas as pd
import numpy as np
import torch
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from sklearn.metrics import classification_report
from scipy.special import softmax

def load_data(file_path):
    data = []
    with open(file_path, 'r') as f:
        for line in f:
            if line.strip() == '':
                continue
            text = line.strip()
            aspect = next(f).strip()

            sentiment = next(f).strip()
            data.append({'text': text, 'sentiment': sentiment, 'aspect': aspect})

    texts = [d['text'] for d in data]
    sentiments = [d['sentiment'] for d in data]
    aspects = [d['aspect'] for d in data]

    for i, text in enumerate(texts):
        texts[i] = text.replace("$T$", sentiments[i])

    sentiment_dict = {'Positive': 2, 'Neutral': 1, 'Negative': 0}
    sentiments = [sentiment_dict[sentiment] for sentiment in sentiments]
    data_dict = {'text': texts, 'aspect': aspects, 'sentiment': sentiments}
    df = pd.DataFrame(data_dict)

    return df

train_df = load_data('restaurants_train_turkish.xml.dat')
test_df = load_data('restaurants_test_turkish.xml.dat')

# BERT Tokenizer'ı yükleyin
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

# Verilerinizi BERT modeline uygun hale getirin
def prepare_data(dataframe):
    input_ids = []
    attention_masks = []
    labels = []

    for index, row in dataframe.iterrows():
        encoded_dict = tokenizer.encode_plus(
            row['text'],
            add_special_tokens=True,
            max_length=64,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt',
        )
        input_ids.append(encoded_dict['input_ids'])
        attention_masks.append(encoded_dict['attention_mask'])
        labels.append(row['sentiment'])

    input_ids = torch.cat(input_ids, dim=0)
    attention_masks = torch.cat(attention_masks, dim=0)
    labels = torch.tensor(labels)

    return input_ids, attention_masks, labels

train_input_ids, train_attention_masks, train_labels = prepare_data(train_df)
test_input_ids, test_attention_masks, test_labels = prepare_data(test_df)

# Veri yükleyicilerini oluşturun
batch_size = 32

train_data = TensorDataset(train_input_ids, train_attention_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

test_data = TensorDataset(test_input_ids, test_attention_masks, test_labels)
test_sampler = SequentialSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

def train_model(model, train_dataloader, epochs=4):
    optimizer = AdamW(model.parameters(), lr=2e-5, eps=1e-8)
    for epoch in range(epochs):
        total_loss = 0
        model.train()
        for step, batch in enumerate(train_dataloader):
            b_input_ids = batch[0].to(device)
            b_input_mask = batch[1].to(device)
            b_labels = batch[2].to(device)
            model.zero_grad()
            outputs = model(b_input_ids,
                            token_type_ids=None,
                            attention_mask=b_input_mask,
                            labels=b_labels)
            loss = outputs.loss
            logits = outputs.logits
            total_loss += loss.item()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
        avg_train_loss = total_loss / len(train_dataloader)

# BERT modelini yükleyin
model = BertForSequenceClassification.from_pretrained(
    "bert-base-multilingual-cased",
    num_labels=3,
    output_attentions=False,
    output_hidden_states=False,
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Modeli eğitin
train_model(model, train_dataloader)

def evaluate_model(model, dataloader):
    model.eval()
    predictions , true_labels = [], []
    for batch in dataloader:
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        with torch.no_grad():
            outputs = model(b_input_ids,
                            token_type_ids=None,
                            attention_mask=b_input_mask)
        logits = outputs.logits
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        predictions.append(logits)
        true_labels.append(label_ids)
    predictions = [softmax(logits, axis=1) for logits in predictions]
    predictions = np.concatenate(predictions, axis=0)
    predictions = np.argmax(predictions, axis=1)
    true_labels = np.concatenate(true_labels, axis=0)
    print(classification_report(true_labels, predictions))

# Eğitim verilerini kullanarak modelinizin performansını değerlendirin
evaluate_model(model, train_dataloader)

# Test verilerini kullanarak modelinizin performansını değerlendirin
evaluate_model(model, test_dataloader)

# Hiperparametre optimizasyonu
learning_rates = [2e-5, 3e-5, 5e-5]
epochs = [2, 3, 4]

best_acc = 0
best_lr = None
best_epoch = None

for lr in learning_rates:
    for epoch in epochs:
        # Modeli yeniden yükleyin
        model = BertForSequenceClassification.from_pretrained(
            "bert-base-multilingual-cased",
            num_labels=3,
            output_attentions=False,
            output_hidden_states=False,
        )
        model.to(device)

        # Modeli eğitin
        train_model(model, train_dataloader, epochs=epoch)

        # Modeli değerlendirin
        model.eval()
        predictions , true_labels = [], []
        for batch in test_dataloader:
            batch = tuple(t.to(device) for t in batch)
            b_input_ids, b_input_mask, b_labels = batch
            with torch.no_grad():
                outputs = model(b_input_ids,
                                token_type_ids=None,
                                attention_mask=b_input_mask)
            logits = outputs.logits
            logits = logits.detach().cpu().numpy()
            label_ids = b_labels.to('cpu').numpy()
            predictions.append(logits)
            true_labels.append(label_ids)
        predictions = [softmax(logits, axis=1) for logits in predictions]
        predictions = np.concatenate(predictions, axis=0)
        predictions = np.argmax(predictions, axis=1)
        true_labels = np.concatenate(true_labels, axis=0)

        # Performansı kaydedin
        acc = (predictions == true_labels).mean()
        if acc > best_acc:
            best_acc = acc
            best_lr = lr
            best_epoch = epoch

print(f"En iyi doğruluk: {best_acc}")
print(f"En iyi öğrenme oranı: {best_lr}")
print(f"En iyi epoch sayısı: {best_epoch}")


Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       504
           1       1.00      1.00      1.00       111
           2       1.00      1.00      1.00       770

    accuracy                           1.00      1385
   macro avg       1.00      1.00      1.00      1385
weighted avg       1.00      1.00      1.00      1385

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        46
           1       1.00      1.00      1.00         6
           2       1.00      1.00      1.00        94

    accuracy                           1.00       146
   macro avg       1.00      1.00      1.00       146
weighted avg       1.00      1.00      1.00       146



Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual

En iyi doğruluk: 1.0
En iyi öğrenme oranı: 2e-05
En iyi epoch sayısı: 2


In [2]:
pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 58.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 28.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 119.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 77.7 MB/s eta 0:00:00
